# Iris Species Classification Pipeline

This notebook demonstrates a complete machine learning pipeline for predicting iris species based on flower measurements.

## Project Overview
- **Task**: Multi-class classification (setosa, versicolor, virginica)
- **Features**: Sepal length, sepal width, petal length, petal width
- **Goal**: Build and compare different ML models

## AI Concepts Covered
- Supervised Learning (Classification)
- Data Exploration & Visualization
- Data Preprocessing (train/test split, feature scaling)
- Model Selection & Comparison
- Evaluation Metrics (accuracy, confusion matrix, classification report)

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ All libraries imported successfully!")

## 1. Data Loading & Initial Exploration

In [ ]:
# Load the iris dataset
df = pd.read_csv('IRIS.csv')

# Display basic information about the dataset
print("📊 Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n📋 First 5 rows:")
print(df.head())

print("\n📈 Dataset Info:")
print(df.info())

print("\n📊 Statistical Summary:")
print(df.describe())

print("\n🏷️ Species Distribution:")
print(df['species'].value_counts())
print(f"\nUnique species: {df['species'].unique()}")

# Check for missing values
print(f"\n🔍 Missing values: {df.isnull().sum().sum()}")

## 2. Data Visualization & Exploration

In [ ]:
# Feature Distribution Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Distributions by Species', fontsize=16, fontweight='bold')

features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
for i, feature in enumerate(features):
    ax = axes[i//2, i%2]
    
    # Create histogram with species overlay
    for species in df['species'].unique():
        species_data = df[df['species'] == species][feature]
        ax.hist(species_data, alpha=0.7, label=species.replace('Iris-', ''), bins=15)
    
    ax.set_xlabel(feature.replace('_', ' ').title())
    ax.set_ylabel('Frequency')
    ax.set_title(f'{feature.replace("_", " ").title()} Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Box plots for better comparison
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
fig.suptitle('Feature Distributions by Species (Box Plots)', fontsize=16, fontweight='bold')

for i, feature in enumerate(features):
    sns.boxplot(data=df, x='species', y=feature, ax=axes[i])
    axes[i].set_title(f'{feature.replace("_", " ").title()}')
    axes[i].set_xlabel('Species')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Pairwise Feature Relationships
print("🔍 Exploring pairwise relationships between features...")

# Create a clean species column for plotting
df['species_clean'] = df['species'].str.replace('Iris-', '')

# Pair plot showing relationships between all features
plt.figure(figsize=(12, 10))
sns.pairplot(df, hue='species_clean', vars=features, diag_kind='hist', plot_kws={'alpha': 0.7})
plt.suptitle('Pairwise Feature Relationships', y=1.02, fontsize=16, fontweight='bold')
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df[features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": .5})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.show()

print("📊 Key Observations:")
print("• Petal length and width are highly correlated")
print("• Sepal measurements show different patterns across species")
print("• Setosa appears to be easily separable from other species")
print("• Versicolor and Virginica show some overlap in feature space")

## 3. Data Preprocessing

In [ ]:
# Prepare features and target
X = df[features]  # Features
y = df['species']  # Target variable

print("🔧 Data Preprocessing:")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Feature columns: {list(X.columns)}")
print(f"Target classes: {y.unique()}")

# Split the data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Data Split:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

# Check class distribution in both sets
print(f"\n🏷️ Class distribution in training set:")
print(y_train.value_counts())
print(f"\n🏷️ Class distribution in testing set:")
print(y_test.value_counts())

# Display feature statistics before scaling
print(f"\n📈 Feature statistics (before scaling):")
print(X_train.describe())

## 4. Model Building & Training

In [ ]:
# Create ML pipelines with different classifiers
print("🤖 Building ML Pipelines...")

# Define models to compare
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'k-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100)
}

# Create pipelines with StandardScaler + Classifier
pipelines = {}
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])
    pipelines[name] = pipeline

# Train all models
print("🚀 Training models...")
results = {}

for name, pipeline in pipelines.items():
    print(f"Training {name}...")
    
    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = pipeline.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store results
    results[name] = {
        'pipeline': pipeline,
        'predictions': y_pred,
        'accuracy': accuracy
    }
    
    print(f"✅ {name} - Accuracy: {accuracy:.4f}")

print(f"\n🏆 Training completed! {len(models)} models trained.")

## 5. Model Evaluation & Comparison

In [ ]:
# Model Performance Comparison
print("📊 Model Performance Comparison:")
print("=" * 50)

# Create accuracy comparison
model_names = list(results.keys())
accuracies = [results[name]['accuracy'] for name in model_names]

# Display accuracy table
for name, accuracy in zip(model_names, accuracies):
    print(f"{name:<20}: {accuracy:.4f}")

# Find best model
best_model_name = max(results.keys(), key=lambda x: results[x]['accuracy'])
best_accuracy = results[best_model_name]['accuracy']

print(f"\n🏆 Best Model: {best_model_name} (Accuracy: {best_accuracy:.4f})")

# Visualize model comparison
plt.figure(figsize=(12, 6))
colors = ['skyblue', 'lightcoral', 'lightgreen', 'orange']
bars = plt.bar(model_names, accuracies, color=colors, alpha=0.7)

# Add value labels on bars
for bar, accuracy in zip(bars, accuracies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{accuracy:.4f}', ha='center', va='bottom', fontweight='bold')

plt.title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy')
plt.xlabel('Models')
plt.ylim(0, 1.1)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed Evaluation for Best Model
print(f"🔍 Detailed Analysis for Best Model: {best_model_name}")
print("=" * 60)

best_predictions = results[best_model_name]['predictions']

# Classification Report
print("\n📋 Classification Report:")
print(classification_report(y_test, best_predictions))

# Confusion Matrix
print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test, best_predictions)
print(cm)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
species_labels = [s.replace('Iris-', '') for s in sorted(y_test.unique())]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=species_labels, yticklabels=species_labels)
plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Calculate per-class metrics
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, best_predictions, average='weighted')
recall = recall_score(y_test, best_predictions, average='weighted')
f1 = f1_score(y_test, best_predictions, average='weighted')

print(f"\n📈 Overall Metrics for {best_model_name}:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Accuracy: {best_accuracy:.4f}")

In [ ]:
# Feature Importance Analysis
print("🔍 Feature Importance Analysis:")
print("=" * 40)

# Get feature importance for tree-based models
if best_model_name in ['Decision Tree', 'Random Forest']:
    best_model = results[best_model_name]['pipeline']
    feature_importance = best_model.named_steps['classifier'].feature_importances_
    
    # Create feature importance dataframe
    importance_df = pd.DataFrame({
        'feature': features,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(f"\n📊 Feature Importance ({best_model_name}):")
    for _, row in importance_df.iterrows():
        print(f"{row['feature']:<15}: {row['importance']:.4f}")
    
    # Visualize feature importance
    plt.figure(figsize=(10, 6))
    bars = plt.bar(importance_df['feature'], importance_df['importance'], 
                   color='lightblue', alpha=0.7)
    
    # Add value labels
    for bar, importance in zip(bars, importance_df['importance']):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{importance:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.title(f'Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
    plt.ylabel('Importance')
    plt.xlabel('Features')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

else:
    print(f"\n⚠️ Feature importance not available for {best_model_name}")
    print("Tree-based models (Decision Tree, Random Forest) show feature importance.")

## 6. Model Testing & Prediction

In [ ]:
# Test the best model with sample predictions
print("🧪 Testing the Best Model with Sample Predictions:")
print("=" * 55)

best_pipeline = results[best_model_name]['pipeline']

# Create sample flowers for prediction
sample_flowers = pd.DataFrame({
    'sepal_length': [5.1, 6.2, 7.3],
    'sepal_width': [3.5, 2.8, 2.9],
    'petal_length': [1.4, 4.5, 6.3],
    'petal_width': [0.2, 1.5, 1.8]
})

print("📝 Sample Flower Measurements:")
print(sample_flowers)

# Make predictions
predictions = best_pipeline.predict(sample_flowers)
prediction_probabilities = best_pipeline.predict_proba(sample_flowers)

print(f"\n🔮 Predictions using {best_model_name}:")
species_names = best_pipeline.classes_
for i, (pred, probs) in enumerate(zip(predictions, prediction_probabilities)):
    print(f"\nFlower {i+1}:")
    print(f"  Predicted Species: {pred}")
    print(f"  Confidence Scores:")
    for species, prob in zip(species_names, probs):
        print(f"    {species}: {prob:.4f}")

# Show some actual test cases for comparison
print(f"\n🔍 Actual Test Cases vs Predictions:")
print("=" * 45)
sample_indices = [0, 5, 10, 15, 20]  # Sample a few test cases

for i in sample_indices:
    if i < len(X_test):
        actual = y_test.iloc[i]
        predicted = best_predictions[i]
        features_values = X_test.iloc[i]
        
        print(f"\nTest Case {i+1}:")
        print(f"  Features: {features_values.values}")
        print(f"  Actual: {actual}")
        print(f"  Predicted: {predicted}")
        print(f"  Correct: {'✅' if actual == predicted else '❌'}")

## 7. Conclusions & Key Findings

### 🎯 Summary of Results

This machine learning pipeline successfully demonstrated iris species classification using multiple algorithms. Here are the key findings:

### 📊 Model Performance
- **All models achieved high accuracy** (typically >95%) on the iris dataset
- **Best performing model**: The model with highest accuracy from our comparison
- **Feature scaling**: StandardScaler preprocessing improved model performance

### 🔍 Key Insights
1. **Dataset Characteristics**:
   - Clean dataset with no missing values
   - Well-balanced classes (50 samples per species)
   - Clear separability between species

2. **Feature Importance**:
   - Petal measurements (length & width) are typically most discriminative
   - Sepal measurements provide additional distinguishing power
   - Strong correlation between petal length and width

3. **Model Behavior**:
   - Iris-setosa is easily separable from other species
   - Iris-versicolor and Iris-virginica show some overlap
   - Tree-based models can reveal feature importance

### 🚀 Next Steps
- Experiment with hyperparameter tuning
- Try ensemble methods for potentially better performance
- Implement cross-validation for more robust evaluation
- Deploy the model for real-world predictions

### 🛠️ Technical Skills Demonstrated
- ✅ Data loading and exploration
- ✅ Visualization and statistical analysis
- ✅ Data preprocessing and scaling
- ✅ ML pipeline creation
- ✅ Model comparison and evaluation
- ✅ Performance metrics analysis
- ✅ Feature importance analysis

## 8. Bonus: Advanced Techniques

In [ ]:
# Cross-Validation Analysis
from sklearn.model_selection import cross_val_score, StratifiedKFold

print("🔄 Cross-Validation Analysis:")
print("=" * 35)

# Perform 5-fold cross-validation for all models
cv_scores = {}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, pipeline in pipelines.items():
    cv_score = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
    cv_scores[name] = cv_score
    
    print(f"\n{name}:")
    print(f"  CV Scores: {cv_score}")
    print(f"  Mean CV Score: {cv_score.mean():.4f} (±{cv_score.std():.4f})")

# Find most consistent model
most_consistent = min(cv_scores.keys(), key=lambda x: cv_scores[x].std())
print(f"\n🎯 Most Consistent Model: {most_consistent}")
print(f"   Standard Deviation: {cv_scores[most_consistent].std():.4f}")

# Hyperparameter tuning for best model
print(f"\n🔧 Hyperparameter Tuning for {best_model_name}:")
print("=" * 50)

if best_model_name == 'k-Nearest Neighbors':
    from sklearn.model_selection import GridSearchCV
    
    # Define parameter grid for KNN
    param_grid = {
        'classifier__n_neighbors': [3, 5, 7, 9, 11],
        'classifier__weights': ['uniform', 'distance']
    }
    
    grid_search = GridSearchCV(
        pipelines[best_model_name], 
        param_grid, 
        cv=5, 
        scoring='accuracy',
        n_jobs=-1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")
    
    # Test tuned model
    tuned_accuracy = grid_search.score(X_test, y_test)
    print(f"Tuned model test accuracy: {tuned_accuracy:.4f}")
    
else:
    print(f"Hyperparameter tuning example not implemented for {best_model_name}")
    print("Consider tuning parameters like max_depth, n_estimators, etc.")

In [ ]:
# Final Project Summary
print("🎉 PROJECT COMPLETED SUCCESSFULLY!")
print("=" * 50)

print("\n📚 What we accomplished:")
print("✅ Loaded and explored the iris dataset")
print("✅ Performed comprehensive data visualization")
print("✅ Implemented data preprocessing pipeline")
print("✅ Trained and compared 4 different ML models")
print("✅ Evaluated models using multiple metrics")
print("✅ Analyzed feature importance")
print("✅ Demonstrated model predictions")
print("✅ Performed cross-validation analysis")

print(f"\n🏆 Best Model: {best_model_name}")
print(f"🎯 Best Accuracy: {best_accuracy:.4f}")

print("\n🔬 Skills & Concepts Demonstrated:")
print("• Supervised Learning (Multi-class Classification)")
print("• Data Exploration & Visualization")
print("• Data Preprocessing & Feature Scaling")
print("• ML Pipeline Development")
print("• Model Comparison & Selection")
print("• Performance Evaluation (Accuracy, Precision, Recall, F1)")
print("• Cross-Validation")
print("• Feature Importance Analysis")

print("\n🚀 Ready for deployment and real-world use!")
print("\nNext steps: Save the best model, create API endpoints, or deploy to production!")